# Sprint 5: Emotion Classification Model

**Цель:** Accuracy > 70% на 4 классах эмоций

**Подход:** Два варианта:
1. **Image-based**: EfficientNet на изображениях (простой)
2. **Keypoint-based**: MLP на keypoints features (научный)

**Классы эмоций:**
- 0: sad
- 1: angry
- 2: relaxed
- 3: happy

**Датасет:** HuggingFace Dewa/Dog_Emotion_Dataset_v2 (4000 изображений)

## 1. Установка зависимостей

In [ ]:
!pip install -q datasets transformers timm pillow scikit-learn matplotlib

## 2. Загрузка датасета

In [ ]:
from datasets import load_dataset
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
import json

# Загрузка датасета
print("Загрузка датасета с HuggingFace...")
dataset = load_dataset("Dewa/Dog_Emotion_Dataset_v2")
print(f"Train: {len(dataset['train'])} images")
print(f"Test: {len(dataset['test'])} images")

In [ ]:
# Проверка структуры
print("\nПример записи:")
sample = dataset['train'][0]
print(f"  Label: {sample['label']}")
print(f"  Emotion: {sample['emotion']}")
print(f"  Image size: {sample['image'].size}")

# Классы эмоций
EMOTION_CLASSES = ['sad', 'angry', 'relaxed', 'happy']
NUM_CLASSES = len(EMOTION_CLASSES)

# Статистика по классам
print("\nРаспределение классов (train):")
from collections import Counter
train_labels = [x['label'] for x in dataset['train']]
for label, count in sorted(Counter(train_labels).items()):
    print(f"  {EMOTION_CLASSES[label]}: {count}")

In [ ]:
# Визуализация примеров
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
for i, emotion in enumerate(EMOTION_CLASSES):
    # Находим примеры для каждого класса
    examples = [x for x in dataset['train'] if x['emotion'] == emotion][:2]
    for j, ex in enumerate(examples):
        ax = axes[j, i]
        ax.imshow(ex['image'])
        ax.set_title(f"{emotion.upper()}")
        ax.axis('off')
plt.tight_layout()
plt.savefig('emotion_examples.png', dpi=150)
plt.show()

## 3. Вариант A: Image-based Classification (EfficientNet)

Простой и эффективный подход - fine-tuning CNN на изображениях.

In [ ]:
# Dataset для PyTorch
class EmotionImageDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.data = hf_dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        image = item['image'].convert('RGB')
        label = item['label']
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Transforms
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Создание датасетов
train_dataset = EmotionImageDataset(dataset['train'], train_transform)
test_dataset = EmotionImageDataset(dataset['test'], val_transform)

# DataLoaders
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Train batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")

In [ ]:
import timm

# Создание модели
def create_emotion_model(model_name='efficientnet_b0', num_classes=4):
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes)
    return model

# Устройство
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Модель
model = create_emotion_model('efficientnet_b0', NUM_CLASSES)
model = model.to(device)

# Loss и optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

print(f"\nМодель: EfficientNet-B0")
print(f"Параметры: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Функции обучения
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc='Training'):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return total_loss / len(loader), 100. * correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Evaluating'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return total_loss / len(loader), 100. * correct / total, all_preds, all_labels

In [ ]:
# Обучение
EPOCHS = 20
best_acc = 0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("=" * 60)
print("ОБУЧЕНИЕ IMAGE-BASED МОДЕЛИ")
print("=" * 60)

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc, _, _ = evaluate(model, test_loader, criterion, device)
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Epoch {epoch+1}/{EPOCHS}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'emotion_image_best.pt')
        print(f"  → Сохранена лучшая модель: {val_acc:.2f}%")

print(f"\n{'='*60}")
print(f"ЛУЧШАЯ ACCURACY: {best_acc:.2f}%")
print(f"{'='*60}")

In [ ]:
# График обучения
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(history['train_loss'], label='Train')
ax1.plot(history['val_loss'], label='Validation')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(history['train_acc'], label='Train')
ax2.plot(history['val_acc'], label='Validation')
ax2.axhline(y=70, color='r', linestyle='--', label='Target (70%)')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('emotion_training_curve.png', dpi=150)
plt.show()

In [ ]:
# Финальная оценка
model.load_state_dict(torch.load('emotion_image_best.pt'))
_, final_acc, preds, labels = evaluate(model, test_loader, criterion, device)

print("\n" + "=" * 60)
print("ФИНАЛЬНАЯ ОЦЕНКА (Image-based Model)")
print("=" * 60)
print(f"\nTest Accuracy: {final_acc:.2f}%")
print(f"Target: 70%")
print(f"Status: {'PASS ✓' if final_acc >= 70 else 'FAIL ✗'}")

print("\nClassification Report:")
print(classification_report(labels, preds, target_names=EMOTION_CLASSES))

# Confusion matrix
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap='Blues')
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(EMOTION_CLASSES))
plt.xticks(tick_marks, EMOTION_CLASSES, rotation=45)
plt.yticks(tick_marks, EMOTION_CLASSES)
for i in range(len(EMOTION_CLASSES)):
    for j in range(len(EMOTION_CLASSES)):
        plt.text(j, i, str(cm[i, j]), ha='center', va='center')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.tight_layout()
plt.savefig('emotion_confusion_matrix.png', dpi=150)
plt.show()

## 4. Сохранение результатов

In [ ]:
# Сохранение метрик
metrics = {
    'model_type': 'image_based',
    'architecture': 'efficientnet_b0',
    'num_classes': NUM_CLASSES,
    'emotion_classes': EMOTION_CLASSES,
    'best_val_accuracy': best_acc,
    'final_test_accuracy': final_acc,
    'epochs': EPOCHS,
    'target_accuracy': 70.0,
    'passed': final_acc >= 70.0,
}

with open('emotion_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Сохранённые файлы:")
print("  - emotion_image_best.pt (веса модели)")
print("  - emotion_metrics.json (метрики)")
print("  - emotion_training_curve.png (график обучения)")
print("  - emotion_confusion_matrix.png (матрица ошибок)")

In [ ]:
# Копирование в /kaggle/working/ для скачивания
import shutil
import os

output_dir = '/kaggle/working/'
if os.path.exists(output_dir):
    for f in ['emotion_image_best.pt', 'emotion_metrics.json', 
              'emotion_training_curve.png', 'emotion_confusion_matrix.png',
              'emotion_examples.png']:
        if os.path.exists(f):
            shutil.copy(f, output_dir)
            print(f"Скопировано: {f}")
else:
    print("Локальный режим - файлы в текущей директории")

## 5. Итоги

### Image-based Model (EfficientNet-B0)
- Простой и эффективный подход
- Использует transfer learning
- Подходит для production

### Следующие шаги
1. Скачать `emotion_image_best.pt`
2. Добавить в `packages/models/emotion.py`
3. Интегрировать в full_pipeline.py